# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import numpy as np

# TODO: Fill this in based on where you saved the training and testing data

training_file = '../storage/train.p'
testing_file = '../storage/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
s_train = train['sizes']
c_train = train['coords']
y_train_incremented = [] #this will be loaded later
# print(s_train.shape, c_train.shape, s_train[10], c_train[10],s_train[121], c_train[121])

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = X_train.shape[0] #39209

# TODO: Number of testing examples.
n_test = X_test.shape[0] #12630

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.max(y_test) #42

img_maxPixel = np.max(X_train)#255
cols = image_shape[0]
rows = image_shape[1]
depth_original = image_shape[2]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
print("Shape of one image (width, height, depth) =", image_shape, "or ({0},{1},{2})".format(cols, rows, depth_original))
print('sizes:',s_train.shape,' cords:',c_train.shape)
print('max pixel value = ',img_maxPixel)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
import csv 

signLabels = []
with open('signnames.csv','rt', encoding='ascii') as signNamesFile:
    signReader = csv.reader(signNamesFile, delimiter=',')
    for row in signReader:
#         print(row[1])
        signLabels.append(row[0] + ')' + row[1])
signLabels = signLabels[1:]        
print(";\n".join(signLabels) )

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline
import random

def printImgs(x, y, idxArray=None, figsize=(12,12), size1=4, size2=4, cmap=None):
    fig = plt.figure(figsize=figsize)
    if idxArray==None:
        idxArray=np.random.randint(len(x), size=(size1*size2) )
    for idx in range( len(idxArray) ):
        plot = fig.add_subplot(4,4,idx+1)
        plot.axis('off')
#         plt.xticks([]),plt.yticks([]) # this does the same as plot.axis('off')
        imgIdx = idxArray[idx]
        plt.title(signLabels[ y[imgIdx]] )
        plt.imshow(x[imgIdx].squeeze(), interpolation="nearest", cmap=cmap)
        


In [ ]:
#     print some images
printImgs(X_train, y_train )
printImgs(X_train, y_train, size1=1, size2=1, figsize=(20,20), idxArray=[5000] )

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
# training setup

import tensorflow as tf

In [ ]:
# Definition of preProcess function (algorithm) and constants, so I can call it on training set and test images
# using in-place modification of the numpy array 
import cv2
from skimage import exposure

IMAGE_DEPTH = 1 # 3 if not colored, 1 if grayscale

print(X_train.shape)
print(X_test.shape)

PREPROCESS_PARAMS = {'grayscale':True, 'normalize':True, 'doExposure':True, 'doEqualizeHist':False}

def preProcess(in_images, progressInfo=False, grayscale=True, normalize=True, doExposure=True, doEqualizeHist=False):
    outType = np.float32 if normalize else np.uint8
    outImgDimension = 1 if grayscale else 3
    imgDimOne = in_images.shape[1]
    imgDimTwo = in_images.shape[2]
    out_image = np.zeros((len(in_images), imgDimOne , imgDimTwo, outImgDimension), outType)
    
#     for imgIdx in [5000]:
    for imgIdx in range(len(in_images)):
        if(progressInfo and imgIdx%100==0):
            print("preProcessing image {0}".format(imgIdx))
        img=in_images[imgIdx]
        if(grayscale):
            img = cv2.cvtColor(in_images[imgIdx], cv2.COLOR_BGR2GRAY)
            img = img.reshape((imgDimOne , imgDimTwo, 1))
        
#         The algorithm normalizes the brightness and increases the contrast of the image.
        if(doEqualizeHist and grayscale):
            img = cv2.equalizeHist(img)
        
        if(normalize):
            img = img/img_maxPixel
        # local contrast enhancement http://scikit-image.org/docs/dev/api/skimage.exposure.html
        if(doExposure):
            img = exposure.equalize_adapthist(img.squeeze())
        img = img.reshape((imgDimOne , imgDimTwo, outImgDimension))
        out_image[imgIdx]=img    
    return out_image

In [ ]:
# TEST code - Just to PLAY AROUND with image manipulations


### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline


# Grayscale and normalize images through preProcess function.. Notes: fine-tuned the preprocessing,
# using in-place modification of the numpy array 
import cv2
from skimage import exposure

IMAGE_DEPTH = 1 # 3 if not colored, 1 if grayscale

print(X_train.shape)
print(X_test.shape)


fig2 = plt.figure()

X_trainPreprocessed=preProcess(X_train[390:394], **PREPROCESS_PARAMS)
        
print(X_trainPreprocessed.shape)
printImgs(X_trainPreprocessed, y_train[390:394], idxArray=[0,1,2,3], size1=2, size2=2, cmap='gray' )


with open('../storage/modified/testSavePickle.p', 'wb') as handle:
    pickle.dump(X_trainPreprocessed, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../storage/modified/testSavePickle.p','rb') as handle:
    x2 = pickle.load(handle)

print(x2.shape)    
img = x2[0]
plt.figure()

# plt.xticks([]), plt.yticks([])
plt.imshow(img.squeeze(), cmap='gray')
plt.show()

In [ ]:
# playing around
X_trainPreprocessed=preProcess(X_train[0:4], grayscale=True, doEqualizeHist=True, normalize=True, doExposure=False)
img = X_trainPreprocessed[0]
plt.figure()

# plt.xticks([]), plt.yticks([])
plt.imshow(img.squeeze(), cmap='gray')
plt.show()

In [ ]:
# Transformation code - to increase the amount of data with rotated and affine transformed images

im1 = X_train[500]


def transformImage(im1):
    """Does a small random rotation and affine transformation. Learning from: 
    http://docs.opencv.org/3.1.0/da/d6e/tutorial_py_geometric_transformations.html 
    """
    rotationAngle = np.random.randint(-10,10)
    rotationAngle=10
    M1 = cv2.getRotationMatrix2D((cols/2, rows/2),rotationAngle, 1)
    im1 = cv2.warpAffine(im1, M1, (cols, rows))
    pts1 = np.float32([[3,15],[15,15],[32, 25]])
    pts2 = np.float32([[3+np.random.randint(-1,1),15+np.random.randint(-1,1)],[15+np.random.randint(-1,1),\
    15+np.random.randint(-1,1)],[32+np.random.randint(-1,1),25+np.random.randint(-1,1)]])
    M2 = cv2.getAffineTransform(pts1, pts2)
    im1 = cv2.warpAffine(im1, M2, (cols,rows))
    return im1

im1 = transformImage(im1)
plt.imshow(im1);plt.show()

In [ ]:
# function to increment images( increase the amount of data)

def incrementBuildImages(in_images,in_labels, progressInfo=False):
    """transformImage images and appends to the end of the list. The new lists are returned.
    in_images - images to be transformed and added incremented, in_labels - the labels; 
    returns out_images and out_labels with the entire set of images/labels """
    
    out_images = np.empty([0,cols,rows,depth_original]).astype(np.uint8)
    
    out_labels = np.empty([0]).astype(np.uint8)
    out_images = np.append(out_images, in_images, axis=0)
    out_labels = np.append(out_labels, in_labels, axis=0)
    idx = 0;
    
    for im in in_images:
        if progressInfo and idx%50==0 :
            print("incrementBuildImages processing image idx {0}".format(idx))
        #doing 5 random transformations
        for sameImgTransf in range(3):
            imTrans1 = transformImage(im)
            out_images = np.append(out_images, np.reshape(imTrans1, (1,32,32,3)), axis=0)
            out_labels = np.append(out_labels, [in_labels[idx]], axis=0)
        idx += 1;
    return out_images, out_labels




In [ ]:
# TEST CODE - to verify the working of the incrementBuildImages on a small subset of images

newImgs = np.array([X_train[211],X_train[2900], X_train[1900], X_train[25000]])
newLabels = np.array([y_train[211],y_train[2900], y_train[1900], y_train[25000]])
print(newImgs.shape)
newImgs, newLabels = incrementBuildImages(newImgs[0:4], newLabels[0:4], progressInfo = True)
print("shape of ",newImgs.shape, newLabels.shape)
printImgs(newImgs, newLabels, size1=4, figsize=(15,15), size2=4)



In [ ]:
import os
storageTempPath = '../storage/trainAugmentedData'
testPreprocessedFileName = '../storage/testData/testV1SavePickle.p'

PROCESS_STORE_BATCH_SIZE = 1000

def storeAugmentedData(x, y,  progressInfo=True):
    print("### storeAugmentedData called, creating transformed and pre-processed images for augmented data ### ")
    assert len(y) == len(x)
    length = len(x)
    for recIdx in range(0, length, PROCESS_STORE_BATCH_SIZE):
        endRecordIdx = recIdx+PROCESS_STORE_BATCH_SIZE
        if(progressInfo): print("processing record {0} through {1}".format(recIdx, endRecordIdx) )
        newImgs, newLabels = incrementBuildImages(x, y, progressInfo = False)
        newImgs = preProcess(newImgs, progressInfo=False, **PREPROCESS_PARAMS)
        augmImageBatchData = dict(images=newImgs, labels=newLabels)
        fileName = storageTempPath+"/augmDataDump{0}.p".format(recIdx);
        with open(fileName, 'wb') as f:
            pickle.dump(augmImageBatchData, f,  protocol=pickle.HIGHEST_PROTOCOL)
        if(progressInfo): print("batch saved to {0}".format(fileName));
    print("# storeAugmentedData called, creating transformed and pre-processed images for augmented data #")
        
def storeTestData(x):
    X_testTemp = preProcess(x, progressInfo=True, **PREPROCESS_PARAMS)
    with open(testPreprocessedFileName, 'wb') as handle:
        pickle.dump(X_testTemp, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# PREPROCESSING AND TRANSFORMATION - saving and loading the data

X_trainPreprocessed = np.empty([0,32,32,1], np.uint8)
X_testPreprocessed = np.empty([0,32,32,1], np.uint8)
y_train_incremented = np.empty([0], np.uint8)    


if not os.path.exists(storageTempPath):
    os.makedirs(storageTempPath)
if not os.path.exists('../storage/testData/'):
    os.makedirs('../storage/testData/')


# conditions wheather to process or not the data if it already exists
if os.path.exists(storageTempPath+"/augmDataDump0.p") :
    print('augmDataDump0.p exists, so seems like augmented data exists already')
else:
    print('augmented data doesn\'t seem to exists, will start creating it')
    rangeStart = 0; rangeEnd = len(X_train)
    storeAugmentedData(X_train[rangeStart:rangeEnd], y_train[rangeStart:rangeEnd], progressInfo=True)

if os.path.exists(testPreprocessedFileName):
    print("test data {0} file already exists".format(testPreprocessedFileName))
else:
    storeTestData(X_test)




In [ ]:
# Load pre-processed and transformed data, and test data
for filename in os.listdir(storageTempPath):
    with open(os.path.join(storageTempPath,filename), 'rb') as f:
        augmImageBatchData = pickle.load(f)
        images = augmImageBatchData['images']
        labels = augmImageBatchData['labels']
        X_trainPreprocessed = np.append(X_trainPreprocessed, images, axis=0)
        y_train_incremented = np.append(y_train_incremented, labels)

with open(testPreprocessedFileName,'rb') as handle:
    X_testPreprocessed = pickle.load(handle)

print('X_trainPreprocessed', X_trainPreprocessed.shape)
print('y_train_incremented', y_train_incremented.shape)
print('X_testPreprocessed', X_testPreprocessed.shape)
print('y_test', y_test.shape)


In [ ]:
# preprocessing verifications

print("X_trainPreprocessed[9,0,0,:]={0}".format(X_trainPreprocessed[9,0,0,:]))
print('after processing image shapes:', X_trainPreprocessed.shape, X_testPreprocessed.shape)


printImgs(X_trainPreprocessed, y_train_incremented, size1=1, figsize=(15,15), size2=1, idxArray=[100] , cmap='gray')
printImgs(X_testPreprocessed, y_test, size1=1, figsize=(15,15), size2=1, idxArray=[100] , cmap='gray')

print("Sample of normalized data", X_trainPreprocessed[0, 0, 0, :])
print("Max/min normalized data point ", np.max(X_trainPreprocessed),np.min(X_trainPreprocessed))

printImgs(X_trainPreprocessed, y_train_incremented, idxArray=[0,1,2,3], size1=2, size2=2, cmap='gray' )

In [ ]:
from sklearn.utils import shuffle


EPOCHS = 20
# EPOCHS = 4
BATCH_SIZE = 512

In [ ]:
## This code below allows me to get quick results while setting up/troubleshooting the pipeline

splitRatio = 0.75
print(n_train)

def randomValidationSplit(X,y,splitRationParam=splitRatio):
    X, y = shuffle(X, y)
    subsetTrainIdx = int(len(X)*splitRationParam)
    ## split training data into training and validation set
    X_training, X_validation =  X[:subsetTrainIdx], X[subsetTrainIdx:]
    y_training, y_validation = y[:subsetTrainIdx], y[subsetTrainIdx:]
    return (X_training, y_training, X_validation, y_validation)


X_training, y_training, X_validation, y_validation = randomValidationSplit(X_trainPreprocessed, y_train_incremented)


print("X/y training/validation shapes =",X_training.shape, X_validation.shape, y_training.shape, y_validation.shape)

printImgs(X_training, y_training, idxArray=[0,1,2,3], size1=2, size2=2, cmap='gray' )

printImgs(X_validation, y_validation, idxArray=[0,1,2,3], size1=2, size2=2, cmap='gray' )

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_


**Answer:**

I shuffle the data because I noticed that many consecutive images are simmilar which need shuffled especially if I am planning to divide data between validation and training sets. Also made sure to include all the attributes like sizes, labels, etc. 

Then I normalized the data, dividing by 255 (max value). I tried a tensorflow l2_normalize, but it was slow or even failing. 

In [ ]:
### Generate additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

I took about a quarter of the dataset as validation set, and left the training set be first 30,000 images after being shuffled. 

**Answer:**

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

# let's start by repeating LeNet architecture
# input - conv+act - pooling - conv+act - pooling - flatten - fullyConnect+act - fullyConnect
from tensorflow.contrib.layers import flatten


def NNet4(x, dropoutArg):
    mu = 0
    sigma = 0.1
    
    conv0_w = tf.Variable( tf.truncated_normal(shape=(5,5,IMAGE_DEPTH,3), mean=mu, stddev=sigma) )
    conv0_b = tf.Variable(tf.zeros(3))
    conv0 = tf.nn.conv2d(x, conv0_w, strides=[1,1,1,1], padding='SAME')+conv0_b
    conv0 = tf.nn.relu(conv0)
    

    conv1_w = tf.Variable( tf.truncated_normal(shape=(5,5, 3,6), mean=mu, stddev=sigma) )
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(conv0, conv1_w, strides=[1,1,1,1], padding='VALID')+conv1_b
    conv1 = tf.nn.relu(conv1)
    
    #Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    #Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    conv2 = tf.nn.relu(conv2)

    #Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    #Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # Layer 3: Fully Connected. 
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 84), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(84))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Activation.
    fc1    = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropoutArg)
    
    # Layer 5: Fully Connected. Input = 84. Output = 10.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(84, 42), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(42))
    
    logits = tf.matmul(fc1, fc2_W) + fc2_b
    

    return logits
#



# some alternatives I tried:
# 1) Pooling. Input = 28x28x6. Output = 14x14x6.-> #Pooling. Input = 10x10x16. Output = 5x5x16. ->
#   flatten. In=5x5x16,Out= 400-> Fully Connected in=400,out=120 -> relu -> Fully Connected in=120,Out=84 -> Activation

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

I followed closely LeNet architecture, I have 
Convolution - relu - pooling --->

Convolution - relu - pooling --->

fully connected - relu --->

fully connected - relu --->

fully connected to output 

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, IMAGE_DEPTH))
y = tf.placeholder(tf.int32, (None))
learning_rate_plh = tf.placeholder(tf.float32, (None))
one_hot_y = tf.one_hot(y , 42)
keep_prob=tf.placeholder(tf.float32)
ratePlh = tf.placeholder(tf.float32)

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.


rate = 0.0002
rate_1 = 0.001
rate_2 = 0.0002
# rate = 0.001

ratesToMaxEpoch = [(0.01,2),(0.01, 4), (0.001, 6), (0.0001, 1000)]

dropout = 0.5 #not using dropout yet
# logits = NNet(x)
# logits = NNet2(x, keep_prob)
# logits = NNet3(x, keep_prob)
logits = NNet4(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_plh)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy, loss = sess.run([accuracy_operation, loss_operation], feed_dict={x: batch_x, y: batch_y, keep_prob:1})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples, loss

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Training...")
    print()
    rateIdx = 0;
    currentRate = rate;
    for i in range(EPOCHS):
        if i < 5 :
            currentRate = rate_1
        else:
            currentRate = rate_2
            
#         currentRate, maxEp = ratesToMaxEpoch[rateIdx];
#         if(maxEp==i):
#             rateIdx+=1;
#         print('Rate: {0}'.format(currentRate))    

        X_training, y_training, X_validation, y_validation = randomValidationSplit(X_trainPreprocessed[:],
                                                                                   y_train_incremented[:])

    #         printImgs(X_training, y_training, size1=1, figsize=(15,15), size2=1, idxArray=[5000] , cmap='gray')
        for offset in range(0, len(X_training), BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_training[offset:end], y_training[offset:end]
            
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, 
                                                    keep_prob:dropout, learning_rate_plh:currentRate})  #,rate:currentRate 
            
        validation_accuracy, loss = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}, loss={:.3f}".format(validation_accuracy, loss))
        
    test_accuracy, loss = evaluate(X_testPreprocessed, y_test)
    print("Test Accuracy = {:.3f}, loss={:.3f}".format(test_accuracy, loss))
    
    
    saver.save(sess, './trafPredResult')
    print("Model saved")

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_

**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import os

images = np.empty([0,32,32,3]).astype(np.uint8)
filenames = []
computedLabels = np.empty([0,1])
folderName = 'customImg'
for filename in os.listdir(folderName):
    testim = cv2.imread(os.path.join(folderName, filename))
    testim = cv2.cvtColor(testim, cv2.COLOR_BGR2RGB)
    testim = np.reshape(testim, (1,32,32,3))
    images = np.append(images, testim , axis=0)
    filenames.append(filename)

testLabelIdx = None

imagesProc = preProcess(images, **PREPROCESS_PARAMS)


with tf.Session() as sess:
    s = tf.train.latest_checkpoint('.')
    saver.restore(sess, tf.train.latest_checkpoint('.'))

#     print(imagesProc.shape)
    logit = sess.run(logits,feed_dict={x:imagesProc,keep_prob:1.0})
    print(logit.shape)
    v = sess.run(tf.argmax(logit, 1))
    testLabelIdx = v
    print(type(testLabelIdx))
    print('My images - Neural Network results (indexes) =', np.array2string( testLabelIdx, separator=', ') )

print("images shape=", images.shape)
printImgs(images,testLabelIdx, size1=5, size2=1, idxArray=[0,1,2,3,4])



### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**
I plotted the images above, with the labels. 
First when I tried with original data set (without incrementing it extra transformed images), it predicted correctly one or two images.  
The reason I believe is because the images were slightly different. Transfromation should help.
After running on the dataset with the transformation the result is: 
...


In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

the accuracy wasn't as high on my images. I used the lables from signnames.csv to visualize the output and the lable on the same plot, so was able to see if the image matches the label. 

Next, using softmax to compute the results gives me:


In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.